In [44]:
import os
from enum import Enum
import random
from typing import Any, Dict, List

import matplotlib.pyplot as plt
from openai import OpenAI, AsyncOpenAI
import pandas as pd
import pprint
from pydantic import BaseModel, Extra, Field
import tiktoken

from config import settings

In [2]:
openai_client = OpenAI(api_key=settings.openai_api_key)

In [3]:
tokenizer = tiktoken.encoding_for_model("gpt-4o")

In [4]:
def calculate_tokens(text):
    return len(tokenizer.encode(text))
calculate_tokens("hello world")

2

# TVTropes Dataset
* https://github.com/dhruvilgala/tvtropes

In [5]:
# tropes table
tropes_fname = "story/TVTropesData/tropes.csv"
tropes_fpath = os.path.join(settings.data_dir, tropes_fname)

tropes_df = pd.read_csv(tropes_fpath)
print(tropes_df.shape, tropes_df.columns)

(30984, 4) Index(['Unnamed: 0', 'TropeID', 'Trope', 'Description'], dtype='object')


In [6]:
tropes_df.head()

,Unnamed: 0,TropeID,Trope,Description
0,0,t00001,AbandonedArea,\nAbandoned places make good settings for fict...
1,1,t00002,AbandonedCatchphrase,Catchphrases are a great and simple way to hel...
2,2,t00003,AbandonedHospital,The creepy abandoned hospital/mental instituti...
3,3,t00004,AbandonedHospitalAwakening,"An Abandoned Hospital Awakening is, as the nam..."
4,4,t00005,AbandonedInfoPage,When a work is getting more and more complicat...


In [7]:
idx = 1
print("TROPE:")
print(tropes_df.iloc[idx]["Trope"])
print("DESC:")
print(tropes_df.iloc[idx]["Description"].strip())

TROPE:
AbandonedCatchphrase
DESC:
Catchphrases are a great and simple way to help a character's image. But sometimes, catchphrases don't stick. Perhaps the writers were experimenting with one early on, but eventually decided against it. Or perhaps a catchphrase was done to death and is no longer funny. Or sometimes, a character simply evolves and the catchphrase no longer fits the character.
Whatever the reason, this trope is whenever an early catchphrase is dropped or becomes used seldom. This is a catchphrase specific sub-trope of Early Installment Weirdness, and frequently a result of Characterization Marches On and Character Development. The closest inverse to this would be Flanderization (wherein a character's catchphrase acts as a quirk that takes over their personality). Unrelated to Subverted Catch Phrase.


In [8]:
# film-tropes table (tropes in films)
film_tropes_fname = "story/TVTropesData/film_tropes.csv"
film_tropes_fpath = os.path.join(settings.data_dir, film_tropes_fname)

film_tropes_df = pd.read_csv(film_tropes_fpath)
print(film_tropes_df.shape, film_tropes_df.columns)

(751594, 6) Index(['Unnamed: 0', 'Title', 'Trope', 'Example', 'trope_id', 'title_id'], dtype='object')


In [9]:
film_title_ids = film_tropes_df.title_id.unique()
len(film_title_ids), film_title_ids[:5]

(17019, array(['f0', 'f1', 'f2', 'f3', 'f4'], dtype=object))

In [10]:
title_rows = film_tropes_df[film_tropes_df.title_id=='f0']
print(title_rows.shape)

(35, 6)


In [11]:
print("Title:",title_rows.iloc[0]["Title"])
# for i in range(title_rows.shape[0]):
for i in range(5):
    row = title_rows.iloc[i]
    print(row["Trope"])
    print(row["Example"].strip())
    print('-'*30)

Title: ABBATheMovie
MsFanservice
The concert segments make it clear that Agnetha and Frida's looks play a big part of the band's act; the "Get on the Carousel"/"I'm a Marionette" segment has them in form-fitting leotards that have them cut very shapely figures, and Frida at points also wears very short shorts (such as in one performance of "Why Did It Have to Be Me?"), leading to some sensationalist headlines hoping for a  Wardrobe Malfunction . And that's not even counting the repeated mentions about Agnetha's bottom.
------------------------------
InsistentTerminology
The radio station manager that tasks Ashley with delivering an interview with the band insists that it's not an interview, it's a  dialogue , and when Ashley asks if he wants him to do a documentary, he says that it's not a documentary, it's an  event .
------------------------------
TheIngenue
A rare male example; when Benny reads one newspaper reporting about the band asking for a "kinky bed", he ask what "kinky" mean

In [45]:
# film-imdb-match table (tropes in films)
film_imdb_match_fname = "story/TVTropesData/film_imdb_match.csv"
film_imdb_match_fpath = os.path.join(settings.data_dir, film_imdb_match_fname)

film_imdb_match_df = pd.read_csv(film_imdb_match_fpath)
print(film_imdb_match_df.shape, film_imdb_match_df.columns)

(390511, 8) Index(['Unnamed: 0', 'Title', 'Trope', 'Example', 'CleanTitle', 'tconst',
       'trope_id', 'title_id'],
      dtype='object')


In [46]:
film_imdb_match_df.head()

,Unnamed: 0,Title,Trope,Example,CleanTitle,tconst,trope_id,title_id
0,0,ABBATheMovie,MsFanservice,The concert segments make it clear that Agnet...,abbathemovie,tt0075617,t14656,f0
1,1,ABBATheMovie,InsistentTerminology,The radio station manager that tasks Ashley w...,abbathemovie,tt0075617,t11527,f0
2,2,ABBATheMovie,TheIngenue,A rare male example; when Benny reads one new...,abbathemovie,tt0075617,t23019,f0
3,3,ABBATheMovie,GettingCrapPastTheRadar,During Ashley's fantasy sequence in which he ...,abbathemovie,tt0075617,t09016,f0
4,4,ABBATheMovie,WhoWearsShortShorts,"Frida's outfit in the ""Why Did It Have to Be ...",abbathemovie,tt0075617,t25994,f0


# index

In [16]:
from llama_index.core import (
	Document,
	SimpleDirectoryReader, VectorStoreIndex,
	StorageContext,
	Settings,
	QueryBundle
)
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore

from llama_index.embeddings.text_embeddings_inference import (
    TextEmbeddingsInference,
)
from qdrant_client import QdrantClient
from config import app_settings

In [17]:
embedder = TextEmbeddingsInference(
    settings.embedding_model,
    base_url=settings.embedding_base_url,
    auth_token=settings.embedding_api_key
)
Settings.embed_model = embedder

In [18]:
client = QdrantClient(host="localhost", port=app_settings.qdrant_port)

vector_store = QdrantVectorStore(client=client, collection_name="tropes")
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
def make_document(row):
    text = "Trope: {}\n{}".format(
        row['Trope'], row['Description'].strip()
    )
    document = Document(
        text = text,
        metadata = {
            "TropeID": row["TropeID"]
        },
        text_template='{content}'
    )
    return document

documents = [make_document(tropes_df.iloc[i]) for i in range(tropes_df.shape[0])]

In [20]:
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    embed_model = embedder,
    show_progress = True,
    transformations = [],
)

Parsing nodes:   0%|          | 0/30984 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

...

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/962 [00:00<?, ?it/s]

In [21]:
index

In [22]:
retriever_args = {"similarity_top_k": 10}
retriever = index.as_retriever(**retriever_args)

# Query
make a story trope between 2 characters from simschat using o3-mini-high
* "Vivienne LaRoux" (00d66087-9b3b-46da-bd74-bf45cbe81d3c)
    * Female, 28, Style Influencer
* "Zephyr Orion" (35f0c56f-263d-42df-846c-e1833d8ca0ab)
    * Male, 28, Astronaut


## Prompt
```
Given the following 2 character specifications write 3 story tropes that are suitable for the interaction of these characters.
interaction is character 1->character2

[Character1]
{{character1}}
[Character2]
{{character2}}

Return in the following json format
{"tropes": List[str]}

Rules:
* the tropes must be detailed description in minimum 3 sentences.
* the tropes must focus on character behavioral interaction and less about the surrounding settings of the characters
```

In [39]:
# tropes
# try1
# tropes = [
#     "Celestial Charm vs. Urban Chic: Zephyr Orion’s playful, space-faring spirit meets Vivienne LaRoux’s sleek, high-fashion attitude in a clash of lifestyles. His witty, adventurous banter serves as a counterpoint to her assertive and sometimes dismissive sophistication, creating a dynamic tension that is both humorous and endearing. As Zephyr attempts to bridge the cosmic with the couture, both characters are forced to confront their own preconceptions and discover hidden depths beneath their outward facades.",
#     "Stars Align on the Runway: In this trope, Zephyr uses his infectious enthusiasm and captivating space adventure stories to charm Vivienne, whose cool demeanor and stylish insights challenge his every word. Their encounter transforms a simple conversation into a runway where cosmic dreams meet high-fashion realities, revealing unexpected vulnerabilities and shared aspirations. Over time, the playful interjections of Zephyr begin to soften Vivienne’s guarded exterior, sparking a journey where differences illuminate common ground.",
#     "Cosmic Coaxing of a Sophisticated Heart: Here, Zephyr’s jovial, ambitious nature is put to the test as he attempts to win over Vivienne’s sophisticated yet guarded heart. His goofball charm and materialistic bragging about his interstellar exploits clash with her sharp, dismissive remarks, yet gradually reveal moments of genuine connection. Through persistent humor and heartfelt storytelling, Zephyr slowly unravels Vivienne’s noncommittal shell, allowing both characters to experience personal growth and a surprising, unconventional camaraderie."
# ]

# try2
tropes = [
    "Cosmic Charmer Meets Fashionable Realist: Zephyr’s playful banter and captivating tales of space adventures are directed toward Vivienne, whose confident and sometimes sharp-tongued demeanor creates a fascinating counterpoint. He tries to charm her with humor and exuberance, even sharing personal quirks and prized memorabilia in his storytelling. Vivienne, with her intellectual brilliance and stylish assertiveness, responds with measured, sometimes dismissive remarks, sparking a witty verbal dance that reveals both their vulnerabilities and strengths.",
    "Galactic Showdown of Egos: Zephyr’s materialistic flair and outgoing nature lead him to boast about his latest cosmic exploits and treasured possessions, aiming to impress Vivienne. Instead, Vivienne’s cutting wit and high-fashion critique quickly put his self-praise under scrutiny, resulting in a spirited clash of egos. Their interaction evolves into a dynamic battle where humor and sharp retorts collide, highlighting their contrasting values and the tension between ambition and sophistication.",
    "Unlikely Mentor in a Duel of Perspectives: In this trope, Zephyr’s ambition and heartfelt vulnerability shine through as he shares not only his adventures but also his moments of loneliness and introspection. Vivienne’s assertive and sometimes mean responses mask a hidden curiosity about his genuine passion for exploration and self-improvement. Their dialogue becomes a transformative exchange where Zephyr’s open-hearted storytelling and resilient optimism gradually challenge Vivienne’s noncommittal, guarded attitude, paving the way for mutual growth and unexpected understanding."
]
query = tropes[0]
pprint.pprint(query)

('Cosmic Charmer Meets Fashionable Realist: Zephyr’s playful banter and '
 'captivating tales of space adventures are directed toward Vivienne, whose '
 'confident and sometimes sharp-tongued demeanor creates a fascinating '
 'counterpoint. He tries to charm her with humor and exuberance, even sharing '
 'personal quirks and prized memorabilia in his storytelling. Vivienne, with '
 'her intellectual brilliance and stylish assertiveness, responds with '
 'measured, sometimes dismissive remarks, sparking a witty verbal dance that '
 'reveals both their vulnerabilities and strengths.')


In [40]:
results = retriever.retrieve(query)

In [42]:
for result in results:
    trope_id = result.node.metadata["TropeID"]
    print("{} - {:.3f}".format(trope_id, result.score))
    print(result.node.text)
    # pprint.pprint(result.node.text[:200])
    print('-'*30)
    

t29945 - 0.575
Trope: DarkFeminineLightFeminine


This is a character contrast trope. Both female characters are decidedly feminine but in different ways. The light is generally angelic and feels love, whereas the dark is generally devilish and plays with desire. The light is usually sweet and naïve, whereas the dark is usually aloof and brooding. Either type applies to a Tomboy and Girly Girl contrast. When a love interest is involved, it's likely to lead to a Betty and Veronica love triangle: Betty being the Light Feminine, and Veronica being the Dark Feminine.
As with any two-character trope, the two characters have to have some sort of relationship with each other, be it friends, family, love interests, or rivals for a love interest, so the two characters can be playing the traits directly off each other.
The dark feminine character is not always a villain or a whore, or even necessarily half of a "good girl, bad girl" or "naughty and nice" pair. It could be that she's just the mor

In [48]:
result.node.text.split("\n", 1)[0][7:]

'MustMakeHerLaugh'

In [49]:
result.node.text.split("\n", 1)[1].strip()

"He's the joker of the group. She's the Emotionless Girl or perhaps the Tsundere, maybe both. He makes it his mission in life to make her let out one of the most powerful emotional responses. It's a matter of personal pride. If he can break through her shell, he can brag about it for years.\nSaid girl's friends will wonder why she puts up with it. Don't expect his jokes to actually be funny. The comedy really comes from her reaction, or lack thereof. Compare Straight Man and Wise Guy for a similar setup.\nThis is often played as romantic interest. After all, guys like girls that laugh at their jokes and girls like guys who can make them laugh. And even if she doesn't laugh, she does at least listen. If it's meant to become a romantic relationship, don't expect them to resolve it anytime soon.\nMost of the time, what finally makes her laugh wasn't even intended to be funny.\nCan result in a When She Smiles moment, where the guy falls even harder for her when he finally sees her radiant 